In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import numpy as np
from scipy import misc
import sys
sys.path.insert(0,'../libs')
import ruidos as noise
from scipy.signal import convolve2d
%aimport edge_detectors
# import edge_detectors as ed
from image_helpers import *

# Pŕactica *Detección de bordes*

## Ejercicio 1 

In [ ]:
img = misc.imread("lena.png")
print(img.shape)

In [ ]:
lambda_s = [10,20, 30,50]
xi_s = [.1,.2,.4, 1]
gaussian_contaminated_lenas = [noise.add_gaussian_noise(img, 0, l) for l in lambda_s]
rayleigh_contminated_lenas = [noise.mult_rayleigh_noise(img, x) for x in xi_s]

print("Lenas contaminadas con ruido Gaussiano")
show_img_s(gaussian_contaminated_lenas, ["lambda = " + str(l) for l in lambda_s])
print("Lenas contaminadas con ruido de Rayleigh")
show_img_s(rayleigh_contminated_lenas, ["xi = " + str(x) for x in xi_s])

## Ejercicio 2

### Método del Laplaciano

In [ ]:
print("laplaciano ED + lena original")
show_img_s([edge_detectors.laplacian(img)],["laplaciano a lena original"])
print("laplaciano ED + lena's contaminadas con ruido Gaussiano aditivo")
show_img_s([edge_detectors.laplacian(i) for i in gaussian_contaminated_lenas])
print("laplaciano ED + lena's contaminadas con ruido de Rayleigh multiplicativo")
show_img_s([edge_detectors.laplacian(i) for i in rayleigh_contminated_lenas])

### Método del Laplaciano con evaluación local de varianza

Prueba del **LLV**(Laplacian edge detector with Local Variance threshold) sin suavizar la imagen antes de calcular el laplaciano.

In [ ]:
th_range = [45,100,150,200]
show_img_s([edge_detectors.LLV(img, th) for th in th_range], ["TH = " + str(th) for th in th_range])

In [ ]:
show_img_s([edge_detectors.LLV(img, th, _smooth_pre_laplacian=True) for th in th_range], ["TH = " + str(th) for th in th_range])

In [ ]:
SHARPEN_KERNEL = [[0,-1/float(4),0],[-1/float(4),2,-1/float(4)],[0,-1/float(4),0]]
perro_koala = misc.imread("perro_koala.jpg")[:,:,0] # me quedo con el canal rojo
# perro_koala = convolve2d(perro_koala, SHARPEN_KERNEL, mode="same")
show_img(perro_koala)
show_img_s([edge_detectors.LLV(perro_koala, th) for th in th_range], ["TH = " + str(th) for th in th_range])

### **LoG**: *Laplacian of Gaussian* 

In [ ]:
img_test = misc.imread('test.gif')[:,:,0]
img_test_2 = misc.imread('test_2.png')

In [ ]:
sigma_range = [.5,1,2,4]
kern_size = 20

#### LoG kernel's

In [ ]:
show_img_s([edge_detectors._LOG_kern(kern_size, s) for s in sigma_range],\
           ["sigma = " + str(s) for s in sigma_range])

### Gaussian kernel's

In [ ]:
show_img_s([edge_detectors._gaussian_kern(kern_size, s) for s in sigma_range],\
           ["sigma = " + str(s) for s in sigma_range])

In [ ]:
show_img_s([edge_detectors.LOG(img, sig, _kernel_size=kern_size) for sig in sigma_range],\
           ["SIGMA = " + str(sig) for sig in sigma_range])
show_img_s([edge_detectors.LOG(img_test, sig, _kernel_size=kern_size) for sig in sigma_range],\
           ["SIGMA = " + str(sig) for sig in sigma_range])
show_img_s([edge_detectors.LOG(img_test_2, sig, _kernel_size=kern_size) for sig in sigma_range],\
           ["SIGMA = " + str(sig) for sig in sigma_range])

## Ejercicio 3

In [ ]:
show_img_s( [edge_detectors.kirsch_compass(I) for I in [img, img_test, img_test_2]],\
          ["lena", "test 1", "test 2"])

Resulta que en imagenes sin ruido, la obtención de bordes mediante el operador de **Kirsch** funciona mucho mejor. Posiblemente debido a que los métodos que basan su funcionamiento en la segunda derivada suelen ser más sensibles a ruido. Por otro lado, la mejora se ve influenciada debido a que se todo, para cada pixel de la imagen, el *gradiente* (aunque no lo sea exactamente) máximo entre 8 orientaciones diferentes.

A pesar de tener mejores resultados en imagenes sin ruido, tiene una performance aceptable para ruido Gaussiano. Los resultados son mejorables mediante un suavizado, y una umbralización de la imagen resultante.

Por otro lado, el ruido de Rayleigh, posiblemente debido a la drasticidad que le da el hecho que sea multiplicativo en este caso, hace que los resultados de este método se vean drásticamente deteriorados.

### Ruido Gaussiano

In [ ]:
show_img_s([edge_detectors.kirsch_compass(I) for I in gaussian_contaminated_lenas])

### Ruido de Rayleigh

In [ ]:
show_img_s([edge_detectors.kirsch_compass(I) for I in rayleigh_contminated_lenas])